# Full Dataset KMeans

In [1]:
import dask, dask.dataframe as dd, dask.array as da
from dask.diagnostics import ProgressBar
import matplotlib.pyplot as plt
import pandas as pd
import nltk, re, csv, pyarrow, pyarrow
import numpy as np
from dask import delayed, persist
from dask.distributed import Client
from glob import glob

pd.set_option('display.max_columns', None)
csv.field_size_limit(10000000)
csv.field_size_limit()

%matplotlib inline

In [2]:
print("testing")

testing


In [3]:
client = Client(n_workers=3, threads_per_worker=2, memory_limit='2GB')
client

Client Scheduler: tcp://127.0.0.1:55767 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 3 Cores: 6 Memory: 6.00 GB


In [1]:
client.close()

NameError: name 'client' is not defined

In [4]:
x = da.random.random((10000, 10000), chunks='16 MiB')
x

dask.array<random_sample, shape=(10000, 10000), dtype=float64, chunksize=(1250, 1250), chunktype=numpy.ndarray>

In [5]:
%%time

y = (x + x.T) - x.mean(axis=0)
y.sum().compute()

CPU times: user 531 ms, sys: 52.5 ms, total: 584 ms
Wall time: 1.88 s


49994810.48503938

In [6]:
del y
del x

In [7]:
best_list = ['JobID', 'CleanJobTitle', 'CanonCity', 'CanonState', 'JobDate', 'JobText', 'Source', 'CanonEmployer',
             'Latitude', 'Longitude', 'CanonIntermediary', 'CanonJobTitle', 'CanonCounty', 'DivisionCode', 'MSA', 'LMA',
             'InternshipFlag', 'ConsolidatedONET', 'CanonSkillClusters', 'CanonSkills', 'IsDuplicate', 'CanonMinimumDegree', 
             'CanonRequiredDegrees', 'CIPCode', 'MinExperience', 'ConsolidatedInferredNAICS', 'BGTOcc', 'MaxAnnualSalary',
             'MaxHourlySalary', 'MinAnnualSalary', 'MinHourlySalary', 'YearsOfExperience', 'CanonJobHours', 'CanonJobType',
             'CanonPostalCode', 'CanonYearsOfExperienceCanonLevel', 'CanonYearsOfExperienceLevel', 'ConsolidatedTitle', 
             'Language', 'BGTSubOcc', 'ConsolidatedDegreeLevels', 'MaxDegreeLevel', 'MinDegreeLevel']

In [28]:
dtypes={'JobID': np.str, 'CleanJobTitle': np.str, 'JobDomain': np.str, 
        'CanonCity': np.str, 'CanonCountry': np.str, 'CanonState': np.str, 
        'JobText': np.str, 'JobURL': np.str, 'PostingHTML': np.str, 
        'Source': np.str, 'JobReferenceID': np.str, 'Email': np.str, 
        'CanonEmployer': np.str, 'Latitude': np.str, 'Longitude': np.str, 
        'CanonIntermediary': np.str, 'Telephone': np.str, 'CanonJobTitle': np.str, 
        'CanonCounty': np.str, 'DivisionCode': np.str, 'MSA': np.str, 'LMA': np.str,
        'InternshipFlag': np.str, 'ConsolidatedONET': np.str, 'CanonCertification': np.str, 
        'CanonSkillClusters': np.str, 'CanonSkills': np.str, 'IsDuplicate': np.str, 
        'IsDuplicateOf': np.str, 'CanonMaximumDegree': np.str, 'CanonMinimumDegree': np.str, 
        'CanonOtherDegrees': np.str, 'CanonPreferredDegrees': np.str,
        'CanonRequiredDegrees': np.str, 'CIPCode': np.str, 'StandardMajor': np.str, 
        'MaxExperience': np.str, 'MinExperience': np.str, 'ConsolidatedInferredNAICS': np.str, 
        'BGTOcc': np.str, 'MaxAnnualSalary': np.str, 'MaxHourlySalary': np.str, 
        'MinAnnualSalary': np.str, 'MinHourlySalary': np.str, 'YearsOfExperience': np.str, 
        'CanonJobHours': np.str, 'CanonJobType': np.str, 'CanonPostalCode': np.str, 
        'CanonYearsOfExperienceCanonLevel': np.str, 'CanonYearsOfExperienceLevel': np.str, 
        'ConsolidatedTitle': np.str, 'Language': np.str, 'BGTSubOcc': np.str, 'JobDate': np.str,
        'ConsolidatedDegreeLevels': np.str, 'MaxDegreeLevel': np.str, 'MinDegreeLevel': np.str,
                       }

In [58]:
ddf = dd.read_csv('/Volumes/LaCie SSD/bgdata/random_data/samp_20_19/r*.csv', 
                 engine='python', 
                 dtype=dtypes,
                 sample=2_500_000,
#                  encoding='latin-1',
#                  encoding='utf-8',
                 assume_missing=True,
                 error_bad_lines=False,
#                  blocksize=None,
                 usecols=best_list,
#                  parse_dates=['JobDate']
                )
ddf

,JobID,CleanJobTitle,CanonCity,CanonState,JobDate,JobText,Source,CanonEmployer,Latitude,Longitude,CanonIntermediary,CanonJobTitle,CanonCounty,DivisionCode,MSA,LMA,InternshipFlag,ConsolidatedONET,CanonSkillClusters,CanonSkills,IsDuplicate,CanonMinimumDegree,CanonRequiredDegrees,CIPCode,MinExperience,ConsolidatedInferredNAICS,BGTOcc,MaxAnnualSalary,MaxHourlySalary,MinAnnualSalary,MinHourlySalary,YearsOfExperience,CanonJobHours,CanonJobType,CanonPostalCode,CanonYearsOfExperienceCanonLevel,CanonYearsOfExperienceLevel,ConsolidatedTitle,Language,BGTSubOcc,ConsolidatedDegreeLevels,MaxDegreeLevel,MinDegreeLevel
npartitions=73,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [59]:
%%time

# ddf = ddf.repartition(npartitions=70)

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 5.72 µs


In [60]:
%%time

ddf.head()

CPU times: user 71.2 ms, sys: 7.27 ms, total: 78.5 ms
Wall time: 1.49 s


,JobID,CleanJobTitle,CanonCity,CanonState,JobDate,JobText,Source,CanonEmployer,Latitude,Longitude,CanonIntermediary,CanonJobTitle,CanonCounty,DivisionCode,MSA,LMA,InternshipFlag,ConsolidatedONET,CanonSkillClusters,CanonSkills,IsDuplicate,CanonMinimumDegree,CanonRequiredDegrees,CIPCode,MinExperience,ConsolidatedInferredNAICS,BGTOcc,MaxAnnualSalary,MaxHourlySalary,MinAnnualSalary,MinHourlySalary,YearsOfExperience,CanonJobHours,CanonJobType,CanonPostalCode,CanonYearsOfExperienceCanonLevel,CanonYearsOfExperienceLevel,ConsolidatedTitle,Language,BGTSubOcc,ConsolidatedDegreeLevels,MaxDegreeLevel,MinDegreeLevel
0,38596542233,Physical Therapist Home Health,Phoenix,AZ,2019-10-04,Physical Therapist Home Health\n\nLHC Group\n\...,Job Board,LHC Group,33.4507,-112.068,NaN,Physical Therapist,Maricopa,NaN,38060: Metropolitan Statistical Area,MT043806,0,29112300,Health Care: Emergency and Intensive Care;Spec...,{'Cardiopulmonary Resuscitation (CPR)': 'Healt...,FALSE,NaN,NaN,NaN,NaN,621610.0,29-1123.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,85001,NaN,NaN,Physical Therapist,en,Physical Therapist,NaN,NaN,NaN
1,38578970510,Employment/Social Enterprise Specialist,Port Saint Lucie,FL,2019-09-01,Employment/Social Enterprise Specialist\n QR ...,NaN,NaN,27.2981,-80.2967,NaN,NaN,Saint Lucie,NaN,38940: Metropolitan Statistical Area,MT123894,0,21101200,Health Care: Advanced Patient Care;Specialized...,{'Case Management': 'Health Care: Advanced Pat...,FALSE,NaN,NaN,NaN,NaN,NaN,21-1012.92,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34952,NaN,NaN,Employment Enterprise Specialist,en,Career Counselor,NaN,NaN,NaN
2,38639153206,Fso Team Leader,Vernon Hills,IL,2019-10-26,FSO Team Leader\n\n\n\nDate: 21-Oct-2019\n\nLo...,Company,Tetra Pak International S A,42.2284,-87.9674,NaN,NaN,Lake,29404.0,16980: Metropolitan Statistical Area|176: Comb...,DV172940|MT171698,0,35101200,Customer and Client Support: Basic Customer Se...,{'Customer Service': 'Customer and Client Supp...,FALSE,Bachelor's,Bachelor's,141901,7.0,NaN,35-1012.00,NaN,NaN,NaN,NaN,5 years|7 years,fulltime,permanent,60061,6+,high,Team Leader,en,Restaurant / Food Service Supervisor,16,NaN,16.0
3,38484531778,Bookkeeper/Administrative Assistant,Beachwood,OH,2019-01-31,Experienced Bookkeeper/Admin. Asst.\n\nThe Hom...,Job Board,The Home Corporation Of Ohio,41.4515,-81.5098,NaN,Administrative Assistant,Cuyahoga,NaN,17460: Metropolitan Statistical Area,MT391746,0,43601400,Finance: General Accounting;Specialized Skills...,{'Accounting': 'Finance: General Accounting;Sp...,FALSE,NaN,NaN,NaN,NaN,NaN,43-6014.00,37440.0,18.0,28080.0,13.5,3 years,parttime,permanent,44122,NaN,NaN,Administrative Assistant,en,Office / Administrative Assistant,NaN,NaN,NaN
4,38664931302,Gift Maker,Bolingbrook,IL,2019-11-16,Gift Maker\n\nJob Details\n\nJob Order Number\...,Job Board,Premier Employee Solutions,41.6733,-87.9859,NaN,NaN,DuPage,16974.0,16980: Metropolitan Statistical Area|176: Comb...,DV171697|MT171698,0,53706400,NaN,{},FALSE,NaN,NaN,NaN,NaN,NaN,53-7064.00,NaN,NaN,NaN,NaN,NaN,NaN,temporary,60439,NaN,NaN,Maker,en,Packager,NaN,NaN,NaN


In [61]:
%%time

ddf.tail()

CPU times: user 30.7 ms, sys: 3.51 ms, total: 34.2 ms
Wall time: 423 ms


,JobID,CleanJobTitle,CanonCity,CanonState,JobDate,JobText,Source,CanonEmployer,Latitude,Longitude,CanonIntermediary,CanonJobTitle,CanonCounty,DivisionCode,MSA,LMA,InternshipFlag,ConsolidatedONET,CanonSkillClusters,CanonSkills,IsDuplicate,CanonMinimumDegree,CanonRequiredDegrees,CIPCode,MinExperience,ConsolidatedInferredNAICS,BGTOcc,MaxAnnualSalary,MaxHourlySalary,MinAnnualSalary,MinHourlySalary,YearsOfExperience,CanonJobHours,CanonJobType,CanonPostalCode,CanonYearsOfExperienceCanonLevel,CanonYearsOfExperienceLevel,ConsolidatedTitle,Language,BGTSubOcc,ConsolidatedDegreeLevels,MaxDegreeLevel,MinDegreeLevel
4649,38791254071,Registered Nurse/Travel Assignment,Moreno Valley,CA,2020-06-07,Registered Nurse (RN) / Travel Assignment\n\nD...,Job intermediary,NaN,33.8885,-117.222,Accountable Healthcare Staffing,Registered Nurse,Riverside,NaN,40140: Metropolitan Statistical Area,MT064014,0,29114100,Health Care: Basic Living Activities Support;S...,{'Caregiving': 'Health Care: Basic Living Acti...,FALSE,NaN,NaN,NaN,NaN,62.0,29-1141.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,92551,NaN,NaN,Registered Nurse,en,Registered Nurse,NaN,NaN,NaN
4650,38779945336,Assistant Human Resources Manager,Alpharetta,GA,2020-05-15,Assistant HR Manager\n\nFamily Flowers - Alpha...,Job Board,Family Flowers,34.1222,-84.2975,NaN,Human Resources Assistant,Fulton,NaN,12060: Metropolitan Statistical Area,MT131206,0,11312100,Business: Business Strategy;Specialized Skills...,{'Business Strategy': 'Business: Business Stra...,FALSE,NaN,NaN,NaN,NaN,4531.0,11-3121.92,55000.0,26.44,40000.0,19.23,2 years,fulltime,permanent,30004,NaN,NaN,Human Resources Assistant,en,Human Resources Manager,NaN,NaN,NaN
4651,38758065731,Registered Nurse Senior Living,Grand Rapids,MI,2020-04-07,Job Details\n\n*\n\nRegistered Nurse (RN) FULL...,Job Board,Samaritas,42.9664,-85.6552,NaN,Registered Nurse,Kent,NaN,24340: Metropolitan Statistical Area,MT262434,0,29114100,Administration: Administrative Support;Special...,{'Appointment Setting': 'Administration: Admin...,FALSE,Associate's,Associate's|Bachelor's,NaN,1.0,62.0,29-1141.00,NaN,NaN,NaN,NaN,One year,NaN,NaN,49501,0-1,low,Registered Nurse,en,Registered Nurse,14|16,16.0,14.0
4652,38749146523,02 Night Cook & Prep - Gulf Coast Town Center,Fort Myers,FL,2020-03-29,NaN,Job Board,Zaxby's Southwest Florida,26.6285,-81.8674,NaN,Prep Cook,Lee,NaN,15980: Metropolitan Statistical Area,MT121598,0,35201400,Specialized Skills,{'Multi-Tasking': 'Specialized Skills'},FALSE,NaN,NaN,NaN,NaN,722.0,35-2011.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33901,NaN,NaN,Prep Cook,en,Cook,NaN,NaN,NaN
4653,38821741759,Patient Financial Coordinator - Certified Appl...,Des Moines,IA,2020-07-26,Patient Financial Coordinator - Certified Appl...,Education,Methodist College,41.605,-93.6319,NaN,Financial Counselor,Polk,NaN,19780: Metropolitan Statistical Area,MT191978,0,43405103,Human Resources: Compensation and Benefits;Spe...,{'COBRA': 'Human Resources: Compensation and B...,FALSE,Associate's,Associate's|Bachelor's,NaN,5.0,8131.0,43-4051.03,NaN,NaN,NaN,NaN,5 years,fulltime,permanent,50301,1-6,mid,Financial Counselor,en,Registrar / Patient Service Representative,14|16,16.0,14.0


In [62]:
ddf.npartitions

73

In [46]:
%%time

pd.read_csv('/Volumes/LaCie SSD/bgdata/random_data/samp_20_19/rand_samp_20.csv', 
            usecols=['Language'], low_memory=False)['Language'].unique()

CPU times: user 19.3 s, sys: 10.6 s, total: 29.9 s
Wall time: 36.6 s


array(['en', 'es', 'cn', 'de', 'fr', 'xl', 'pt', 'pl', 'it', 'du', 'tr',
       'sv', 'iw'], dtype=object)

In [47]:
pd.read_csv('/Volumes/LaCie SSD/bgdata/random_data/samp_20_19/rand_samp_19.csv', 
            usecols=['Language'], low_memory=False)['Language'].unique()

array(['en', 'es', 'cn', 'xl', 'de', 'fr', 'zh', 'sv', 'it', 'du', 'pl',
       'pt', 'tr'], dtype=object)

In [15]:
%%time

display(ddf.Language.unique().compute())

0             en
1             es
2             cn
3             xl
4             de
5             fr
6           None
7            NaN
8             zh
9             sv
10            it
11            du
12            pl
13           1-6
14            pt
15          16.0
16            tr
17          12.0
18         14|16
19            iw
20    29-1141.00
21     temporary
22            12
Name: Language, dtype: object

CPU times: user 3.85 s, sys: 838 ms, total: 4.68 s
Wall time: 57.4 s


In [16]:
%%time

display(ddf.CanonJobType.unique().compute())

0                            NaN
1                      permanent
2                      temporary
3                           None
4                             en
5                      Associate
6     Senior Benefits Specialist
7                         intern
8          Management Specialist
9                              0
10                          high
11                   Associate's
Name: CanonJobType, dtype: object

CPU times: user 2.68 s, sys: 325 ms, total: 3.01 s
Wall time: 40.4 s


In [34]:
%%time


missing_values = ddf.isna().sum()
missing_count = ((missing_values / ddf.index.size) * 100)
missing_count_pct = missing_count.compute()
missing_count_pct

CPU times: user 12.3 s, sys: 3.6 s, total: 15.9 s
Wall time: 2min 35s


JobID                                0.000000
CleanJobTitle                        0.132982
CanonCity                            0.875664
CanonState                           0.196677
JobDate                              0.157542
JobText                              3.303882
Source                               1.504433
CanonEmployer                       20.691666
Latitude                             0.853001
Longitude                            0.851204
CanonIntermediary                   88.348708
CanonJobTitle                       36.375644
CanonCounty                          0.896829
DivisionCode                        67.929795
MSA                                  3.263648
LMA                                  1.166387
InternshipFlag                       0.164730
ConsolidatedONET                     3.976978
CanonSkillClusters                   5.584142
CanonSkills                          0.165129
IsDuplicate                          0.164930
CanonMinimumDegree                

In [35]:
%time

ddf.map_partitions(len).compute()

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 5.72 µs


0     13274
1     13242
2     13162
3     13297
4     13234
      ...  
68    14708
69    14630
70    14602
71    14504
72     4654
Length: 73, dtype: int64

In [63]:
ddf = ddf[ddf['JobText'].notnull()]

In [37]:
print(f'Missing companies BEFORE adding the Recruitment Agencies: {ddf.CanonEmployer.isna().sum().compute()}')

Missing companies BEFORE adding the Recruitment Agencies: 196729


In [64]:
EmployerClean = ddf['CanonEmployer'].where((ddf['CanonEmployer'].isnull()) & (ddf['CanonIntermediary'].notnull()), 'Recruiting Agency', ddf['CanonEmployer'])

TypeError: where() takes from 2 to 3 positional arguments but 4 were given

In [56]:
ddf = ddf.drop('CanonEmployer', axis=1)
ddf = ddf.assign(EmployerClean=EmployerClean)

In [57]:
ddf['EmployerClean'].head(20)

0     Recruiting Agency
1     Recruiting Agency
2     Recruiting Agency
3     Recruiting Agency
4     Recruiting Agency
5     Recruiting Agency
6     Recruiting Agency
7     Recruiting Agency
8     Recruiting Agency
9                   NaN
10    Recruiting Agency
11    Recruiting Agency
12    Recruiting Agency
13    Recruiting Agency
14    Recruiting Agency
15    Recruiting Agency
16    Recruiting Agency
17    Recruiting Agency
18    Recruiting Agency
19    Recruiting Agency
Name: EmployerClean, dtype: object

In [44]:
EmployerClean[:20]

array(['LHC Group', nan, 'Tetra Pak International S A',
       'The Home Corporation Of Ohio', 'Premier Employee Solutions',
       'CRST International', 'North West Florida State College',
       'Center For Law Social Policy', 'Catholic Health Initiatives',
       'Recruiting Agency', 'Alera Group, Inc',
       'Intermountain Healthcare', 'Covance Incorporated',
       'Kendall Group', 'United Technologies Corporation',
       "Walmart / Sam's", 'Nebraska State Government', 'Thirdchannel',
       'Patients & Purpose', 'Mednax'], dtype=object)

In [ ]:
ddf.loc[(ddf['CanonEmployer'].isna()) & (ddf['CanonIntermediary'].notna()), 'CanonEmployer'] = 'Recruiting Agency'

In [12]:
print(f'Missing companies AFTER adding the Recruitment Agencies: {df.CanonEmployer.isna().sum()}')

Missing companies AFTER adding the Recruitment Agencies: 225015


In [13]:
print('Amount of Unique Employers: ', len(df['CanonEmployer'].unique()))

Amount of Unique Employers:  119789


In [14]:
print('Amount of Unique Jobs: ', len(df['CleanJobTitle'].unique()))

Amount of Unique Jobs:  1000000


In [15]:
print('Percentage of Unique Jobs in the Dataset: %.2f' % (len(df['CleanJobTitle'].unique()) / df.shape[0] * 100))

Percentage of Unique Jobs in the Dataset: 100.00


In [24]:
rows_to_drop = list(missing_count_pct[(missing_count_pct < 10) & (missing_count_pct > 0)].index)
rows_to_drop

['CleanJobTitle',
 'CanonCity',
 'CanonState',
 'JobDate',
 'JobText',
 'Source',
 'Latitude',
 'Longitude',
 'CanonCounty',
 'MSA',
 'LMA',
 'InternshipFlag',
 'ConsolidatedONET',
 'CanonSkillClusters',
 'CanonSkills',
 'IsDuplicate',
 'BGTOcc',
 'CanonPostalCode',
 'ConsolidatedTitle',
 'Language',
 'BGTSubOcc']

In [25]:
ddf_clean2 = ddf_clean1.dropna(subset=rows_to_drop)

CPU times: user 18 ms, sys: 18.7 ms, total: 36.8 ms
Wall time: 42.6 ms


In [ ]:
# ddf = ddf.persist()

In [26]:
%%time


missing_values = ddf_clean2.isna().sum()
missing_count = ((missing_values / ddf_clean2.index.size) * 100)
missing_count_pct = missing_count.compute()
missing_count_pct

CPU times: user 18.7 s, sys: 3.86 s, total: 22.6 s
Wall time: 2min 45s


JobID                                0.000000
CleanJobTitle                        0.000000
CanonCity                            0.000000
CanonState                           0.000000
JobDate                              0.000000
JobText                              0.000000
Source                               0.000000
CanonEmployer                       20.364444
Latitude                             0.000000
Longitude                            0.000000
CanonIntermediary                   87.645489
CanonJobTitle                       33.591365
CanonCounty                          0.000000
DivisionCode                        66.620770
MSA                                  0.000000
LMA                                  0.000000
InternshipFlag                       0.000000
ConsolidatedONET                     0.000000
CanonSkillClusters                   0.000000
CanonSkills                          0.000000
IsDuplicate                          0.000000
CanonMinimumDegree                

In [27]:
ddf_clean2.head()

,JobID,CleanJobTitle,CanonCity,CanonState,JobDate,JobText,Source,CanonEmployer,Latitude,Longitude,CanonIntermediary,CanonJobTitle,CanonCounty,DivisionCode,MSA,LMA,InternshipFlag,ConsolidatedONET,CanonSkillClusters,CanonSkills,IsDuplicate,CanonMinimumDegree,CanonRequiredDegrees,CIPCode,MinExperience,ConsolidatedInferredNAICS,BGTOcc,MaxAnnualSalary,MaxHourlySalary,MinAnnualSalary,MinHourlySalary,YearsOfExperience,CanonJobHours,CanonJobType,CanonPostalCode,CanonYearsOfExperienceCanonLevel,CanonYearsOfExperienceLevel,ConsolidatedTitle,Language,BGTSubOcc,ConsolidatedDegreeLevels,MaxDegreeLevel,MinDegreeLevel
0,38596542233,Physical Therapist Home Health,Phoenix,AZ,2019-10-04,Physical Therapist Home Health\n\nLHC Group\n\...,Job Board,LHC Group,33.4507,-112.068,NaN,Physical Therapist,Maricopa,NaN,38060: Metropolitan Statistical Area,MT043806,0,29112300,Health Care: Emergency and Intensive Care;Spec...,{'Cardiopulmonary Resuscitation (CPR)': 'Healt...,FALSE,NaN,NaN,NaN,NaN,621610.0,29-1123.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,85001,NaN,NaN,Physical Therapist,en,Physical Therapist,NaN,NaN,NaN
2,38639153206,Fso Team Leader,Vernon Hills,IL,2019-10-26,FSO Team Leader\n\n\n\nDate: 21-Oct-2019\n\nLo...,Company,Tetra Pak International S A,42.2284,-87.9674,NaN,NaN,Lake,29404.0,16980: Metropolitan Statistical Area|176: Comb...,DV172940|MT171698,0,35101200,Customer and Client Support: Basic Customer Se...,{'Customer Service': 'Customer and Client Supp...,FALSE,Bachelor's,Bachelor's,141901,7.0,NaN,35-1012.00,NaN,NaN,NaN,NaN,5 years|7 years,fulltime,permanent,60061,6+,high,Team Leader,en,Restaurant / Food Service Supervisor,16,NaN,16.0
3,38484531778,Bookkeeper/Administrative Assistant,Beachwood,OH,2019-01-31,Experienced Bookkeeper/Admin. Asst.\n\nThe Hom...,Job Board,The Home Corporation Of Ohio,41.4515,-81.5098,NaN,Administrative Assistant,Cuyahoga,NaN,17460: Metropolitan Statistical Area,MT391746,0,43601400,Finance: General Accounting;Specialized Skills...,{'Accounting': 'Finance: General Accounting;Sp...,FALSE,NaN,NaN,NaN,NaN,NaN,43-6014.00,37440.0,18.0,28080.0,13.5,3 years,parttime,permanent,44122,NaN,NaN,Administrative Assistant,en,Office / Administrative Assistant,NaN,NaN,NaN
6,38564109700,Multimedia Graphic Designer,Niceville,FL,2019-08-01,Multimedia Graphic Designer\n\n\t\t\t\t\t\t Ca...,Education,North West Florida State College,30.4829,-86.3126,NaN,Multimedia Designer,Okaloosa,NaN,18880: Metropolitan Statistical Area,MT121888,0,15113200,Design: Graphic and Visual Design Software;Spe...,{'Adobe Acrobat': 'Design: Graphic and Visual ...,FALSE,Bachelor's,Bachelor's,NaN,3.0,6113.0,15-1131.00,30225.0,14.53,30225.0,14.53,three to five years,fulltime,permanent,32578,1-6,mid,Multimedia Designer,en,Software Developer / Engineer,16,NaN,16.0
7,38529906917,Special Assistant To The Chief Operating Officer,Washington,DC,2019-05-10,Special Assistant to the Chief Operating Offic...,Company,Center For Law Social Policy,38.9086,-77.018,NaN,Chief Operating Officer,District of Columbia,47894.0,47900: Metropolitan Statistical Area|548: Comb...,DV114789|MT114790,0,11101100,Administration: Administrative Support;Special...,{'Administrative Support': 'Administration: Ad...,FALSE,Bachelor's,Bachelor's,NaN,2.0,NaN,11-1011.00,NaN,NaN,NaN,NaN,minimum of 2 years,NaN,NaN,20001,1-6,mid,Chief Operating Officer,en,Chief Executive Officer,16,NaN,16.0


In [16]:
df['year'] = df['JobDate'].dt.year
print("Observations per Year")
df['year'].value_counts() / df.shape[0] * 100

Observations per Year


2007    11.4916
2011    10.6304
2012     9.3713
2017     8.3910
2015     8.3816
2016     8.1078
2010     7.6323
2013     7.4669
2020     7.4604
2019     7.4288
2018     7.0356
2014     6.6023
Name: year, dtype: float64

In [17]:
%%time

df = df[df['CanonEmployer'].notna()]
df = df[df['JobText'].notna()]
df = df[df['CleanJobTitle'].notna()]

CPU times: user 2.2 s, sys: 3.76 s, total: 5.96 s
Wall time: 7.05 s


In [40]:
cols_to_drop = list(missing_count_pct[missing_count_pct >= 60].index)
cols_to_drop

['DivisionCode',
 'CanonIntermediary',
 'CIPCode',
 'MaxAnnualSalary',
 'MaxHourlySalary',
 'MinAnnualSalary',
 'MinHourlySalary',
 'MaxDegreeLevel']

In [19]:
df['clean_text'] = df.loc[:, 'JobText'].apply(lambda x: ' '.join(list(filter(None, x.split('\n')))), meta=np.str)

In [20]:
df['len_text'] = df.loc[:, 'clean_text'].apply(len, meta=int)

In [21]:
df['low_clean'] = df.loc[:, 'clean_text'].apply(lambda x: x.lower(), meta=np.str)

In [63]:
ddf.tail()

,Language,CanonCity,CanonState,CleanJobTitle,JobDate,JobID,Latitude,Longitude,CanonPostalCode,CanonCounty,DivisionCode,LMA,MSA,CanonEmployer,CanonJobTitle,ConsolidatedONET,CanonIntermediary,InternshipFlag,Source,CanonSkillClusters,CanonSkills,IsDuplicate,CanonMinimumDegree,CanonRequiredDegrees,CIPCode,MinExperience,ConsolidatedInferredNAICS,BGTOcc,MaxAnnualSalary,MaxHourlySalary,MinAnnualSalary,MinHourlySalary,CanonJobHours,CanonJobType,CanonYearsOfExperienceCanonLevel,CanonYearsOfExperienceLevel,ConsolidatedDegreeLevels,ConsolidatedTitle,MaxDegreeLevel,MinDegreeLevel,BGTSubOcc,YearsOfExperience,titles_uni_500,titles_bi_500,titles_uni_1000,titles_bi_1000,dist__bi_1000,dist__uni_500,dist__bi_500,dist__uni_1000,dist__bi_1000_dummy,dist__uni_500_dummy,dist__bi_500_dummy,dist__uni_1000_dummy,clean_text,charac_text_len,len_job_text,occu_text
1555,en,Chicago,IL,Application Architect - .Net,2011-11-07,9.905835e+06,41.8691,-87.6717,60684,Cook,16974.0,DV171697|MT171698,16980: Metropolitan Statistical Area|176: Comb...,Recruiting Agency,.Net Architect,15-119902,Randstad,False,Job intermediary,Information Technology: Microsoft Development ...,{'.NET': 'Information Technology: Microsoft De...,False,Bachelor's in Computer Science,Bachelor's|Bachelor's in Computer Science,110701|140101,10.0,54.0,151133,NaN,NaN,NaN,NaN,fulltime,permanent,6+,high,16,.Net Architect,NaN,16.0,Application Support Engineer / Analyst,10+ years|2+ years,178,411,203,347,3026.0,1174.0,1475.0,2382.0,False,False,False,False,New Homes in Chicago / / Application Architec...,3455,468,"Software Developers, Systems Software"
1556,en,Tampa,FL,Rcm - Accounts Receivable Specialist,2018-01-09,3.831647e+10,27.9827,-82.3402,33601,Hillsborough,NaN,MT124530,45300: Metropolitan Statistical Area,Recruiting Agency,Accounts Receivable Specialist,43-303100,Vaco Staffing,False,Job intermediary,Finance: Accounts Payable and Receivable;Speci...,{'Accounts Payable / Accounts Receivable': 'Fi...,False,Higher Secondary Certificate,Associate's|General Equivalency Degree|Higher ...,NaN,1.0,6221.0,433031,NaN,NaN,NaN,NaN,NaN,NaN,1-6,mid,14|12,Accounts Receivable Specialist,14.0,12.0,Accounts Receivable / Payable Specialist,1 to 3 years|1-3 years,395,127,150,234,3180.0,1273.0,1553.0,2574.0,False,False,False,False,RCM - A/R Specialist $19/HR -Vaco Staffing Ta...,1234,167,"Bookkeeping, Accounting, and Auditing Clerks"
1557,en,Durango,CO,Rcis/Registered Cardiovascular Invasive Specia...,2017-03-02,3.818972e+10,37.3268,-107.831,81301,La Plata,NaN,MC082042,20420: Micropolitan Statistical Area,Centura Health,NaN,29203100,NaN,False,Job Board,Health Care: Emergency and Intensive Care;Spec...,{'Advanced Cardiac Life Support (ACLS)': 'Heal...,False,Bachelor's,Bachelor's,510901,1.0,622110.0,292031,NaN,NaN,NaN,NaN,fulltime,permanent,0-1,low,16,Registered Cardiovascular Specialist,NaN,16.0,Cardiovascular Technician / Technologist,One year|6 months,395,4,110,813,3318.0,1359.0,1624.0,2746.0,False,False,False,False,RCIS/Registered Cardiovascular Invasive Specia...,3921,531,Cardiovascular Technologists and Technicians
1558,en,Frederick,MD,"Process Systems Software Test Engineer - , Doc...",2011-10-04,6.981341e+06,39.4944,-77.4535,21702,Frederick,43524.0,DV241364|MT114790,47900: Metropolitan Statistical Area|548: Comb...,U7 Equipment,Software Test Engineer,15-119902,NaN,False,Job Board,Engineering: Engineering Software;Specialized ...,{'LabVIEW': 'Engineering: Engineering Software...,False,Bachelor's,Bachelor of Science|Bachelor's|Doctor of Medicine,140101|141001|141901,5.0,62.0,151133,100000.0,48.080002,75000.0,36.060001,NaN,NaN,6+,high,16|21,Software Test Engineer,21.0,16.0,Systems Engineer,5-10 years,495,163,919,895,4356.0,1604.0,2150.0,3230.0,False,False,False,False,| | | Employers &raquo; Process Systems So...,3839,557,"Software Developers, Systems Software"
1560,en,Cheyenne,WY,Qsr Assistant Manager,2017-01-05,3.816282e+10,41.1245,-104.84,82001,Laramie,NaN,MT561694,16940: Metropolitan Statistical Area,Kroger Co

In [41]:
ddf = ddf.drop(cols_to_drop, axis=1)

In [42]:
ddf.tail()

,Language,CanonCity,CanonState,CleanJobTitle,JobDate,JobID,Latitude,Longitude,CanonPostalCode,CanonCounty,LMA,MSA,CanonEmployer,CanonJobTitle,ConsolidatedONET,InternshipFlag,Source,CanonSkillClusters,CanonSkills,IsDuplicate,CanonMinimumDegree,CanonRequiredDegrees,MinExperience,ConsolidatedInferredNAICS,BGTOcc,CanonJobHours,CanonJobType,CanonYearsOfExperienceCanonLevel,CanonYearsOfExperienceLevel,ConsolidatedDegreeLevels,ConsolidatedTitle,MinDegreeLevel,BGTSubOcc,YearsOfExperience,titles_uni_500,titles_bi_500,titles_uni_1000,titles_bi_1000,dist__bi_1000,dist__uni_500,dist__bi_500,dist__uni_1000,dist__bi_1000_dummy,dist__uni_500_dummy,dist__bi_500_dummy,dist__uni_1000_dummy,clean_text,charac_text_len,len_job_text,occu_text
1556,en,Tampa,FL,Rcm - Accounts Receivable Specialist,2018-01-09,3.831647e+10,27.9827,-82.3402,33601,Hillsborough,MT124530,45300: Metropolitan Statistical Area,Recruiting Agency,Accounts Receivable Specialist,43-303100,False,Job intermediary,Finance: Accounts Payable and Receivable;Speci...,{'Accounts Payable / Accounts Receivable': 'Fi...,False,Higher Secondary Certificate,Associate's|General Equivalency Degree|Higher ...,1.0,6221.0,433031,NaN,NaN,1-6,mid,14|12,Accounts Receivable Specialist,12.0,Accounts Receivable / Payable Specialist,1 to 3 years|1-3 years,395,127,150,234,3180.0,1273.0,1553.0,2574.0,False,False,False,False,RCM - A/R Specialist $19/HR -Vaco Staffing Ta...,1234,167,"Bookkeeping, Accounting, and Auditing Clerks"
1557,en,Durango,CO,Rcis/Registered Cardiovascular Invasive Specia...,2017-03-02,3.818972e+10,37.3268,-107.831,81301,La Plata,MC082042,20420: Micropolitan Statistical Area,Centura Health,NaN,29203100,False,Job Board,Health Care: Emergency and Intensive Care;Spec...,{'Advanced Cardiac Life Support (ACLS)': 'Heal...,False,Bachelor's,Bachelor's,1.0,622110.0,292031,fulltime,permanent,0-1,low,16,Registered Cardiovascular Specialist,16.0,Cardiovascular Technician / Technologist,One year|6 months,395,4,110,813,3318.0,1359.0,1624.0,2746.0,False,False,False,False,RCIS/Registered Cardiovascular Invasive Specia...,3921,531,Cardiovascular Technologists and Technicians
1558,en,Frederick,MD,"Process Systems Software Test Engineer - , Doc...",2011-10-04,6.981341e+06,39.4944,-77.4535,21702,Frederick,DV241364|MT114790,47900: Metropolitan Statistical Area|548: Comb...,U7 Equipment,Software Test Engineer,15-119902,False,Job Board,Engineering: Engineering Software;Specialized ...,{'LabVIEW': 'Engineering: Engineering Software...,False,Bachelor's,Bachelor of Science|Bachelor's|Doctor of Medicine,5.0,62.0,151133,NaN,NaN,6+,high,16|21,Software Test Engineer,16.0,Systems Engineer,5-10 years,495,163,919,895,4356.0,1604.0,2150.0,3230.0,False,False,False,False,| | | Employers &raquo; Process Systems So...,3839,557,"Software Developers, Systems Software"
1559,en,New York,NY,Expert Programmer,2019-04-11,3.851693e+10,40.694,-73.9898,11201,Kings,DV363564|MT363562,35620: Metropolitan Statistical Area|408: Comb...,Logicsoft,NaN,15113100,False,Job Board,NaN,{},False,NaN,NaN,8.0,NaN,151131,fulltime,temporary,6+,high,NaN,Programmer,NaN,Computer Programmer,8 years,340,274,414,170,2470.0,1063.0,1189.0,2166.0,True,False,True,False,Logicsoft Logo M204 expert programmer Applied ...,715,110,Computer Programmers
1560,en,Cheyenne,WY,Qsr Assistant Manager,2017-01-05,3.816282e+10,41.1245,-104.84,82001,Laramie,MT561694,16940: Metropolitan Statistical Area,Kroger Company,Assistant Manager,41101100,False,Job Board,Finance: Budget Management;Specialized Skills|...,{'Budgeting': 'Finance: Budget Management;Spec...,False,Higher Secondary Certificate,NaN,2.0,445110.0,411011,fulltime,permanent,1-6,mid,12,Assistant Manager,12.0,Assistant Retail Store Manager,2-3 years,115,187,81,93,2786.0,1129.0,1352.0,2306.0,False,False,False,False,Part-Time QSR Assistant Manager Job Type : Pe...,4488,664,First-Line Supervisors of Retail Sales Workers


## Supervised Task

Rhe vars below can be classified throughout the dataset.

In [46]:
ddf['occu_text'] = ddf['occu_text'].fillna('Unknown')
ddf['JobDate'] = dd.to_datetime(ddf['JobDate'])
ddf['CanonYearsOfExperienceLevel'] = ddf['CanonYearsOfExperienceLevel'].fillna('Unknown')
ddf = ddf.drop(['CanonYearsOfExperienceCanonLevel'], axis=1)

In [44]:
ddf.head()

,Language,CanonCity,CanonState,CleanJobTitle,JobDate,JobID,Latitude,Longitude,CanonPostalCode,CanonCounty,LMA,MSA,CanonEmployer,CanonJobTitle,ConsolidatedONET,InternshipFlag,Source,CanonSkillClusters,CanonSkills,IsDuplicate,CanonMinimumDegree,CanonRequiredDegrees,MinExperience,ConsolidatedInferredNAICS,BGTOcc,CanonJobHours,CanonJobType,CanonYearsOfExperienceCanonLevel,CanonYearsOfExperienceLevel,ConsolidatedDegreeLevels,ConsolidatedTitle,MinDegreeLevel,BGTSubOcc,YearsOfExperience,titles_uni_500,titles_bi_500,titles_uni_1000,titles_bi_1000,dist__bi_1000,dist__uni_500,dist__bi_500,dist__uni_1000,dist__bi_1000_dummy,dist__uni_500_dummy,dist__bi_500_dummy,dist__uni_1000_dummy,clean_text,charac_text_len,len_job_text,occu_text
0,en,Allentown,PA,Advertising Account Executive Key Accounts,2016-07-08,3.808799e+10,40.6023,-75.4707,18101,Lehigh,MT421090,10900: Metropolitan Statistical Area,Morning Call,Advertising Account Executive,41301100,False,Job Board,Marketing and Public Relations: Advertising;Sp...,{'Advertising': 'Marketing and Public Relation...,False,Bachelor's,Bachelor's,2.0,NaN,413011,fulltime,permanent,1-6,mid,16,Advertising Account Executive,16.0,Advertising Account Executive,At least 2-4 years|2 to 3 years,156,56,995,286,3632.0,1371.0,1783.0,2770.0,False,False,False,False,Advertising Account Executive (Key Accounts) T...,4633,684,Advertising Sales Agents
1,en,Hanover,NJ,Crop Protection Sales Specialist For Bayer Cro...,2018-06-14,3.837958e+10,40.8218,-74.422,07981,Morris,DV343508|MT363562,35620: Metropolitan Statistical Area|408: Comb...,Bayer Corporation,Sales Specialist,41401200,False,Company,Finance: Budget Management;Specialized Skills|...,{'Budgeting': 'Finance: Budget Management;Spec...,False,NaN,NaN,NaN,325412.0,414011,NaN,NaN,NaN,NaN,NaN,Sales Specialist,NaN,Sales Representative,NaN,0,486,581,196,4452.0,1815.0,2200.0,3652.0,False,False,False,False,. Crop Protection Sales Specialist for Ba...,3266,435,"Sales Representatives, Wholesale and Manufactu..."
2,en,Milwaukee,WI,Program And Policy Chief Performance Improveme...,2017-03-27,3.820203e+10,43.0343,-87.9151,53201,Milwaukee,MT553334,33340: Metropolitan Statistical Area,Wisdot,NaN,11919900,False,Job Board,Finance: Budget Management;Specialized Skills|...,{'Budgeting': 'Finance: Budget Management;Spec...,False,NaN,NaN,NaN,NaN,119199,fulltime,permanent,NaN,NaN,NaN,Chief Program Performance Manager,NaN,Program Officer,NaN,313,158,58,607,3852.0,1509.0,1898.0,3054.0,False,False,False,False,In partnership with Hispanic Professionals of ...,5141,740,"Managers, All Other"
3,en,West Caldwell,NJ,Machine Operator/Nutraceutical,2020-07-17,3.881592e+10,40.8069,-74.2778,07006,Essex,DV343508|MT363562,35620: Metropolitan Statistical Area|408: Comb...,Vitaquest,Machine Operator,51919900,False,Job Board,Science and Research: Biopharmaceutical Manufa...,{'Batch Records': 'Science and Research: Bioph...,False,Higher Secondary Certificate,Higher Secondary Certificate,1.0,NaN,519012,fulltime,permanent,0-1,low,12,Machine Operator,12.0,Manufacturing Machine Operator,Minimum of 1 year,212,227,653,642,2668.0,1062.0,1293.0,2162.0,False,False,False,False,Machine Operator / Nutraceutical Vitaquest - W...,3417,516,"Separating, Filtering, Clarifying, Precipitati..."
4,en,Houston,TX,Sales Process Coordinator - Midmarket,2018-04-25,3.835360e+10,30.0683,-95.2163,77325,Harris,MT482642,26420: Metropolitan Statistical Area,Recruiting Agency,Sales Coordinator,41401200,False,Job intermediary,Business: Business Consulting;Specialized Skil...,{'Business Consulting': 'Business: Business Co...,False,Higher Secondary Certificate,Higher Secondary Certificate,2.0,NaN,414011,NaN,NaN,1-6,mid,12,Sales Coordinator,12.0,Sales Coordinator,One to two-years,315,232,327,756,3160.0,1261.0,1544.0,2554.0,False,False,False,False,Sales Process Coordinator - MidMarket - Job Ti...,3045,452,"Sales Representatives, Wholesale and Manufactu..."


In [47]:
%%time


missing_values = ddf.isna().sum()
missing_count = ((missing_values / ddf.index.size) * 100)
missing_count_pct = missing_count.compute()
missing_count_pct

CPU times: user 12.2 s, sys: 2.78 s, total: 15 s
Wall time: 2min 11s


Language                        0.000000
CanonCity                       1.363583
CanonState                      0.014230
CleanJobTitle                   0.000000
JobDate                         0.000000
JobID                           0.000000
Latitude                        1.358933
Longitude                       1.358933
CanonPostalCode                 1.668761
CanonCounty                     1.405851
LMA                             1.663126
MSA                             2.956401
CanonEmployer                   0.000000
CanonJobTitle                  42.674160
ConsolidatedONET                4.666445
InternshipFlag                  0.000000
Source                          3.839532
CanonSkillClusters              2.736042
CanonSkills                     0.000000
IsDuplicate                     0.000000
CanonMinimumDegree             39.107542
CanonRequiredDegrees           45.810485
MinExperience                  45.578572
ConsolidatedInferredNAICS      14.844142
BGTOcc          

In [ ]:
from dask.dask_ml import 